In [ ]:
import pandas as pd
import numpy as np
import json
import os
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio
init_notebook_mode(connected=True)

In [ ]:
component_to_analyze = "example/"

os.system("radon cc -j {0} > {0}radon_cc.json".format(component_to_analyze))
os.system("radon mi -j {0} > {0}radon_mi.json".format(component_to_analyze))
os.system("radon raw -j {0} > {0}radon_raw.json".format(component_to_analyze))
os.system("pylint -f json {0} > {0}pylint.json".format(component_to_analyze))
os.system("pylint -r y {0} > {0}pylint.txt".format(component_to_analyze))

radon_cc_filename = "{}radon_cc.json".format(component_to_analyze)
radon_mi_filename = "{}radon_mi.json".format(component_to_analyze)
radon_raw_filename = "{}radon_raw.json".format(component_to_analyze)
pylint_filename = "{}pylint.json".format(component_to_analyze)

In [ ]:
def append_component_cc(df, components_dict):
    for key, value in components_dict.items():
            sum_cc = 0
            sub_parts = 0
            for j in value:
                sum_cc += j["complexity"]
                sub_parts += 1
            df = df.append({'path': key,'sum_cc': sum_cc, 'sub_parts': sub_parts}, ignore_index=True)
    return df
            
def create_df_cc():
    with open(radon_cc_filename, "r") as f:
        radon_cc_json = json.load(f)
        df_cc = pd.DataFrame()
        df_cc = append_component_cc(df_cc, radon_cc_json)
        df_cc = df_cc.set_index('path')
        df_cc['cc'] = df_cc['sum_cc'] / df_cc['sub_parts']
        
        return df_cc

In [ ]:
df_cc = create_df_cc()

In [ ]:
df_mi = pd.read_json(radon_mi_filename, orient="index")

In [ ]:
df_raw = pd.read_json(radon_raw_filename, orient="index")

In [ ]:
df_pylint = pd.read_json(pylint_filename, orient="columns")
df_pylint = df_pylint.drop(columns=['column', 'line', 'module', 'obj', 'symbol'])
df_pylint = pd.get_dummies(df_pylint, columns=['message-id', 'type'])
df_pylint = df_pylint.groupby(['path']).sum()
if 'type_error' not in df_pylint.columns:
    df_pylint['type_error']= np.nan
    
if 'type_convention' not in df_pylint.columns:
    df_pylint['type_convention']= np.nan

if 'type_warning' not in df_pylint.columns:
    df_pylint['type_warning']= np.nan
    
if 'type_refactor' not in df_pylint.columns:
    df_pylint['type_refactor']= np.nan
    
if 'type_fatal' not in df_pylint.columns:
    df_pylint['type_fatal']= np.nan

In [ ]:
# Combine dataframes
df_complete = pd.concat([df_cc, df_mi, df_raw, df_pylint], axis=1, sort=True)

In [ ]:
df_complete

In [ ]:
df_complete.describe()

In [ ]:
print(df_complete.sum())

In [ ]:
#######################################################################################
#Plotting

In [ ]:
#######################################################################################
#Raw Metrics Total
trace1 = go.Bar(
    y=['LOC'],
    x=[df_complete['blank'].sum()],
    name='B',
    orientation = 'h'
)
trace2 = go.Bar(
    y=['LOC'],
    x=[df_complete['single_comments'].sum()],
    name='SC',
    orientation = 'h'
)
trace3 = go.Bar(
    y=['LOC'],
    x=[df_complete['multi'].sum()],
    name='MC',
    orientation = 'h'
)
trace4 = go.Bar(
    y=['LOC'],
    x=[df_complete['sloc'].sum()],
    name='SLOC',
    orientation = 'h'
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='stack',
    title='Raw Metrics Total',
    xaxis=dict(
        title='Number of Code Lines'),
    autosize=False,
    width=900,
    height=350,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
pio.write_image(fig, "{}RawMetricsTotal.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Raw Metrics Detailed
trace1 = go.Bar(
    x = df_complete.index,
    y = df_complete['blank'],
    name='B'
)
trace2 = go.Bar(
    x = df_complete.index,
    y = df_complete['single_comments'],
    name='SC'
)

trace3 = go.Bar(
    x = df_complete.index,
    y = df_complete['multi'],
    name='MC'
)

trace4 = go.Bar(
    x = df_complete.index,
    y = df_complete['sloc'],
    name='SLOC'
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='stack',
    title='Raw Metrics Detailed',
    yaxis=dict(
        title='Number of Code Lines'),
    autosize=False,
    width=900,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=300,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
pio.write_image(fig, "{}RawMetricsDetailed.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Cyclomatic Complexity
trace = go.Scatter(
    x = df_complete.index,
    y = df_complete['cc'],
    mode='markers',
    marker=dict(
        colorbar=dict(
                title='SLOC'
            ),
        size=df_complete['sloc']/7.5,
        color = df_complete['sloc'],
        colorscale='Viridis',
        showscale=True
    )
)

data = [trace]

layout = go.Layout(
    title='Cyclomatic Complexity',
    yaxis=dict(
        title='CC'),
    autosize=False,
    width=900,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=300,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
pio.write_image(fig, "{}CyclomaticComplexity.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Maintainability Index
trace = go.Scatter(
    x = df_complete.index,
    y = df_complete['mi'],
    mode='markers',
    marker=dict(
        colorbar=dict(
                title='SLOC'
            ),
        size=df_complete['sloc']/7.5,
        color = df_complete['sloc'],
        colorscale='Viridis',
        showscale=True
    )
)

data = [trace]

layout = go.Layout(
    title='Maintainability Index',
    yaxis=dict(
        title='MI'),
    autosize=False,
    width=900,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=300,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
pio.write_image(fig, "{}MaintainabilityIndex.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Pylint Total
trace1 = go.Bar(
    y=['total'],
    x=[df_complete['type_convention'].sum()],
    name='Convention',
    orientation = 'h'
)
trace2 = go.Bar(
    y=['total'],
    x=[df_complete['type_refactor'].sum()],
    name='Refactor',
    orientation = 'h'
)
trace3 = go.Bar(
    y=['total'],
    x=[df_complete['type_warning'].sum()],
    name='Warning',
    orientation = 'h'
)
trace4 = go.Bar(
    y=['total'],
    x=[df_complete['type_error'].sum()],
    name='Error',
    orientation = 'h'
)
trace5 = go.Bar(
    y=['total'],
    x=[df_complete['type_fatal'].sum()],
    name='Fatal',
    orientation = 'h'
)

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    barmode='stack',
    title='Pylint Total',
    xaxis=dict(
        title='Number of Messages'),
    autosize=False,
    width=900,
    height=350,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='marker-h-bar')
pio.write_image(fig, "{}PylintTotal.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Pylint Detailed
trace1 = go.Bar(
    x = df_complete.index,
    y = df_complete['type_convention'],
    name='Convention'
)
trace2 = go.Bar(
    x = df_complete.index,
    y = df_complete['type_refactor'],
    name='Refactor'
)

trace3 = go.Bar(
    x = df_complete.index,
    y = df_complete['type_warning'],
    name='Warning'
)

trace4 = go.Bar(
    x = df_complete.index,
    y = df_complete['type_error'],
    name='Error'
)
trace5 = go.Bar(
    x = df_complete.index,
    y = df_complete['type_fatal'],
    name='Fatal'
)

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    barmode='stack',
    title='Pylint Detailed',
    yaxis=dict(
        title='Number of Messages'),
    autosize=False,
    width=900,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=300,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
pio.write_image(fig, "{}PylintDetailed.png".format(component_to_analyze))

In [ ]:
#######################################################################################
#Experimental

In [ ]:
x = df_complete.fillna(value={'cc': 0})['cc']
data = [x]
group_labels = ['cc dist']
colors = ['rgb(0, 200, 200)']

fig = ff.create_distplot(data, group_labels, colors=colors, bin_size=0.5)
iplot(fig)

In [ ]:
x = df_complete.dropna()['mi']
data = [x]
group_labels = ['cc dist']
colors = ['rgb(0, 200, 200)']

fig = ff.create_distplot(data, group_labels, colors=colors, bin_size=5)
iplot(fig)

In [ ]:
fig = {
    "data": [{
        "type": 'violin',
        "y": df_complete.dropna()['mi'],
     "legendgroup": 'M',
            "scalegroup": 'M',
            "name": 'M',
            "box": {
                "visible": True
            },
            "points": 'all',
            "jitter": 0,
            "scalemode": 'count',
            "meanline": {
                "visible": True
            },
            "line": {
                "color": '#8dd3c7'
            },
            "marker": {
                "line": {
                    "width": 2,
                    "color": '#8dd3c7'
                }
            },
            "span": [
                0
            ],
    }],
    "layout" : {
        "title": "",
        "yaxis": {
            "zeroline": False,
        }
    }
}

iplot(fig, filename = 'violin/basic', validate = False)